In [1]:
import pandas as pd
import time
import calendar
import ccxt
import os
import numpy as np
from dotenv import load_dotenv
from alpaca_trade_api.rest import REST 
from alpaca_trade_api.rest import TimeFrame
from alpaca_trade_api.rest import TimeFrameUnit
import random
import krakenex
from pykrakenapi import KrakenAPI
from datetime import datetime
import questionary
import requests
import json
import functions

In [2]:
load_dotenv()

ALPACA_API_KEY_ID = os.getenv('ALPACA_API_KEY_ID')
ALPACA_API_SECRET_KEY = os.getenv('ALPACA_API_SECRET_KEY')

KRAKEN_API_KEY_ID =  os.getenv('KRAKEN_API_KEY_ID')
KRAKEN_API_SECRET_KEY = os.getenv('KRAKEN_API_SECRET_KEY')

In [102]:
def diagnostics(): 
    headers = {'X-CoinAPI-Key' : 'DABB5DFC-447C-4F57-8600-147722F5C7BA'}
    url = 'https://rest.coinapi.io/v1/symbols'

    response = requests.get(url, headers=headers)
    print(response)

    print(response.headers)

    data = json.loads(response.content)  
    df = pd.DataFrame(data)
    return df

symbols_df = diagnostics()

<Response [200]>
{'date': 'Mon, 11 Apr 2022 21:15:03 GMT', 'content-type': 'application/json; charset=utf-8', 'transfer-encoding': 'chunked', 'content-encoding': 'br', 'vary': 'Accept-Encoding', 'x-concurrencylimit-limit': '10', 'x-concurrencylimit-remaining': '10', 'x-ratelimit-used': '283', 'x-ratelimit-overage': 'disabled', 'x-ratelimit-limit': '10000', 'x-ratelimit-remaining': '9717', 'x-ratelimit-reset': '2022-04-12T21:15:03.2367831Z', 'x-ratelimit-request-cost': '1'}


In [104]:
symbols_df.to_excel('symbols.xlsx')  

In [135]:
#get crypto data
crypto_OHLC_df = {}
def getCryptoOHLCData(tickers):
   
    headers = {'X-CoinAPI-Key' : 'DABB5DFC-447C-4F57-8600-147722F5C7BA'}
    
    for ticker in tickers:
        #url = f'https://rest.coinapi.io/v1/exchangerate/{ticker}/USD/history?period_id=1DAY&time_start=2019-01-01T00:00:00&time_end=2022-02-01T00:00:00&limit=100000'
        url = f'https://rest.coinapi.io/v1/ohlcv/{ticker}/USD/history?period_id=1DAY&time_start=2019-01-01T00:00:00&time_end=2022-02-01T00:00:00&limit=10000'

        response = requests.get(url, headers=headers)
        #print(response)
        #print(response.headers)
        data = json.loads(response.content)
        
        df = pd.DataFrame(data)

        ticker_ohlc = df
        ticker_ohlc['symbol'] = ticker
        crypto_OHLC_df[ticker] = ticker_ohlc.reset_index()
        time.sleep(1)
    return crypto_OHLC_df

In [136]:


tickers = ['BTC','ETH','LUNA','SOL','XRP','ADA','AVAX','DOT','DOGE','SHIB','MATIC', 'DAI','LTC','ATOM','LINK','UNI','TRX','BCH']

data = getCryptoOHLCData(tickers)

In [140]:
crypto_tickers = ['BTC','ETH','LUNA','SOL','XRP','ADA','AVAX','DOT','DOGE','SHIB','MATIC', 'DAI','LTC','ATOM','LINK','UNI','TRX','BCH']


all_ohlc = pd.concat(data.values(), join="inner")#.reset_index().set_index("time_close").pivot(columns = 'symbol').dropna()
all_ohlc = all_ohlc.drop(['time_period_start','time_open','time_close','price_open','price_high','price_low', 'trades_count', 'volume_traded'], axis=1)
all_ohlc['time_period_end'] = pd.to_datetime(all_ohlc['time_period_end'])
all_ohlc['time_period_end'] = all_ohlc['time_period_end'].dt.strftime('%Y/%m/%d')
all_ohlc = all_ohlc.drop(['index'], axis=1).set_index('time_period_end').pivot(columns = 'symbol').fillna(0)
all_ohlc.to_csv('ohlc_crypto.csv')
all_ohlc


price_close                                                    \
symbol                  ADA   ATOM   AVAX     BCH       BTC       DAI    DOGE   
time_period_end                                                                 
2019/01/02         0.042100   0.00   0.00  163.66   3826.10  0.000000  0.0000   
2019/01/03         0.045274   0.00   0.00  170.10   3886.20  0.000000  0.0000   
2019/01/04         0.042251   0.00   0.00  159.88   3787.57  0.000000  0.0000   
2019/01/05         0.043808   0.00   0.00  159.39   3820.82  0.000000  0.0000   
2019/01/06         0.044145   0.00   0.00  159.58   3797.41  0.000000  0.0000   
...                     ...    ...    ...     ...       ...       ...     ...   
2022/01/28         1.042800  30.00  64.73  289.64  37188.37  0.999998  0.1415   
2022/01/29         1.048600  29.50  67.06  296.79  37741.90  0.999855  0.1415   
2022/01/30         1.060300  29.10  71.95  299.03  38191.10  0.999723  0.1430   
2022/01/31         1.035900  27.19  67.98  290.04  37901.35  1.000030  0.1394   
2022/02/01         1.052400  28.07  69.92  285.23  38491.93  0.999857  0.1417   

                                                                            \
symbol             DOT      ETH      LINK     LTC   LUNA   MATIC      SHIB   
time_period_end                                                              
2019/01/02        0.00   139.75   0.00000   31.68   0.00  0.0000  0.000000   
2019/01/03        0.00   153.51   0.00000   32.94   0.00  0.0000  0.000000   
2019/01/04        0.00   147.23   0.00000   31.60   0.00  0.0000  0.000000   
2019/01/05        0.00   153.24   0.00000   32.02   0.00  0.0000  0.000000   
2019/01/06        0.00   153.85   0.00000   34.48   0.00  0.0000  0.000000   
...                ...      ...       ...     ...    ...     ...       ...   
2022/01/28       18.13  2426.00  15.23000  106.92  54.18  1.5890  0.000021   
2022/01/29       18.34  2546.84  16.16000  109.49  50.43  1.6740  0.000021   
2022/01/30       18.75  2603.74  16.22304  110.84  50.92  1.6969  0.000022   
2022/01/31       18.19  2602.89  17.77000  108.59  46.32  1.6043  0.000021   
2022/02/01       19.39  2689.72  17.16000  109.55  52.11  1.6429  0.000021   

                                                    
symbol             SOL       TRX      UNI      XRP  
time_period_end                                     
2019/01/02        0.00  0.000000   0.0000  0.36400  
2019/01/03        0.00  0.000000   0.0000  0.37160  
2019/01/04        0.00  0.000000   0.0000  0.35416  
2019/01/05        0.00  0.000000   0.0000  0.35630  
2019/01/06        0.00  0.000000   0.0000  0.34950  
...                ...       ...      ...      ...  
2022/01/28       89.55  0.056453  10.3200  0.60876  
2022/01/29       91.35  0.057631  10.6637  0.61150  
2022/01/30       96.25  0.060341  11.1540  0.61706  
2022/01/31       93.31  0.058692  10.7969  0.60181  
2022/02/01       99.52  0.059132  11.7714  0.61740  

[1127 rows x 18 columns]

In [19]:
all_ohlc.columns = crypto_tickers
all_ohlc_pct = all_ohlc.pct_change()

In [144]:
# Random Crypto
def random_portfolios_generator(tickers, n):
    random_portfolios = pd.DataFrame({'ticker':tickers})
    for n in range(0, n):
        random_portfolios[f'rnd_weight_{n+1}'] = [np.random.rand() for i in range (0, len(tickers))]
        random_portfolios[f'rnd_weight_{n+1}'] = random_portfolios[f'rnd_weight_{n+1}'] / sum(random_portfolios[f'rnd_weight_{n+1}'])
    return(random_portfolios)

random_crypto = random_portfolios_generator(crypto_tickers, 10)


random_crypto

,ticker,rnd_weight_1,rnd_weight_2,rnd_weight_3,rnd_weight_4,rnd_weight_5,rnd_weight_6,rnd_weight_7,rnd_weight_8,rnd_weight_9,rnd_weight_10
0,BTC,0.050822,0.077744,0.079734,0.049157,0.057132,0.027155,0.081628,0.114099,0.023506,0.003702
1,ETH,0.030973,0.055233,0.053645,0.024598,0.016161,0.078359,0.013064,0.017732,0.062563,0.061052
2,LUNA,0.017999,0.028996,0.086526,0.089252,0.067179,0.058663,0.051953,0.102137,0.016421,0.076402
3,SOL,0.025043,0.064300,0.065449,0.067629,0.015178,0.061605,0.063831,0.052531,0.074404,0.034942
4,XRP,0.020051,0.025687,0.068991,0.070652,0.062501,0.029215,0.016114,0.048935,0.099886,0.034240
5,ADA,0.062875,0.039809,0.039299,0.075806,0.090587,0.084627,0.020473,0.002842,0.071745,0.046247
6,AVAX,0.107998,0.020588,0.065688,0.066736,0.023731,0.028737,0.044621,0.099973,0.062957,0.009871
7,DOT,0.037726,0.022729,0.035366,0.033390,0.069935,0.057777,0.068622,0.036869,0.090078,0.029243
8,DOGE,0.056402,0.112653,0.016277,0.056776,0.092899,0.091052,0.018297,0.078081,0.026030,0.094889
9,SHIB,0.107931,0.028920,0.076621,0.052564,0.046028,0.006587,0.022043,0.060844,0.065405,0.030006


In [146]:
# Daily Returns

all_ohlc_pct = all_ohlc.pct_change().fillna(0)
all_ohlc_pct
#random_crypto_daily_returns = all_ohlc.pct_change().dot())
#random_crypto_daily_returns




price_close                                                    \
symbol                  ADA      ATOM      AVAX       BCH       BTC       DAI   
time_period_end                                                                 
2019/01/02         0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
2019/01/03         0.075392  0.000000  0.000000  0.039350  0.015708  0.000000   
2019/01/04        -0.066771  0.000000  0.000000 -0.060082 -0.025380  0.000000   
2019/01/05         0.036851  0.000000  0.000000 -0.003065  0.008779  0.000000   
2019/01/06         0.007693  0.000000  0.000000  0.001192 -0.006127  0.000000   
...                     ...       ...       ...       ...       ...       ...   
2022/01/28        -0.033012 -0.072643 -0.017605  0.001418  0.009439  0.000001   
2022/01/29         0.005562 -0.016667  0.035996  0.024686  0.014884 -0.000143   
2022/01/30         0.011158 -0.013559  0.072920  0.007547  0.011902 -0.000132   
2022/01/31        -0.023012 -0.065636 -0.055177 -0.030064 -0.007587  0.000307   
2022/02/01         0.015928  0.032365  0.028538 -0.016584  0.015582 -0.000173   

                                                                             \
symbol               DOGE       DOT       ETH      LINK       LTC      LUNA   
time_period_end                                                               
2019/01/02       0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
2019/01/03       0.000000  0.000000  0.098462  0.000000  0.039773  0.000000   
2019/01/04       0.000000  0.000000 -0.040909  0.000000 -0.040680  0.000000   
2019/01/05       0.000000  0.000000  0.040820  0.000000  0.013291  0.000000   
2019/01/06       0.000000  0.000000  0.003981  0.000000  0.076827  0.000000   
...                   ...       ...       ...       ...       ...       ...   
2022/01/28      -0.010899  0.004432 -0.015822 -0.000656 -0.000748 -0.078571   
2022/01/29       0.000000  0.011583  0.049810  0.061064  0.024037 -0.069214   
2022/01/30       0.010601  0.022356  0.022341  0.003901  0.012330  0.009716   
2022/01/31      -0.025175 -0.029867 -0.000326  0.095356 -0.020300 -0.090338   
2022/02/01       0.016499  0.065970  0.033359 -0.034328  0.008841  0.125000   

                                                                             
symbol              MATIC      SHIB       SOL       TRX       UNI       XRP  
time_period_end                                                              
2019/01/02       0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  
2019/01/03       0.000000  0.000000  0.000000  0.000000  0.000000  0.020879  
2019/01/04       0.000000  0.000000  0.000000  0.000000  0.000000 -0.046932  
2019/01/05       0.000000  0.000000  0.000000  0.000000  0.000000  0.006042  
2019/01/06       0.000000  0.000000  0.000000  0.000000  0.000000 -0.019085  
...                   ...       ...       ...       ...       ...       ...  
2022/01/28      -0.001320  0.004717 -0.028531  0.005146 -0.024575 -0.017781  
2022/01/29       0.053493  0.000000  0.020101  0.020867  0.033304  0.004501  
2022/01/30       0.013680  0.018779  0.053640  0.047023  0.045978  0.009092  
2022/01/31      -0.054570 -0.027650 -0.030545 -0.027328 -0.032015 -0.024714  
2022/02/01       0.024060  0.016588  0.066552  0.007497  0.090257  0.025905  

[1127 rows x 18 columns]

In [ ]:
#Alpaca 

alpaca_api = REST()

#Facebook, Amazon, Apple, Netflix, Google, Microsoft, Uber, Lyft, Airbnb

FANNGMULA_tickers = ["META", "AMZN", "AAPL", "NTFX", "GOOG", "MSFT", "UBER", "LYFT", "ABNB"]

alpaca_api.get_bars(FANNGMULA_tickers, TimeFrame(45, TimeFrameUnit.Minute), "2021-06-08", "2021-06-08", adjustment='raw').df

,open,high,low,close,volume,trade_count,vwap,symbol
timestamp,,,,,,,,
2021-06-08 07:30:00+00:00,126.10,126.16,125.9600,126.06,20951,304,126.049447,AAPL
2021-06-08 08:15:00+00:00,126.05,126.30,126.0500,126.30,21181,349,126.231904,AAPL
2021-06-08 09:00:00+00:00,126.27,126.32,126.2200,126.28,15955,308,126.284120,AAPL
2021-06-08 09:45:00+00:00,126.29,126.40,125.9000,125.90,30179,582,126.196877,AAPL
2021-06-08 10:30:00+00:00,125.90,126.75,125.8400,126.75,105380,1376,126.530863,AAPL
...,...,...,...,...,...,...,...,...
2021-06-08 20:15:00+00:00,49.81,49.88,49.7800,49.86,21235,97,49.860162,UBER
2021-06-08 21:00:00+00:00,49.86,49.88,49.8600,49.86,44362,30,49.879444,UBER
2021-06-08 21:45:00+00:00,49.86,49.88,49.8000,49.85,1739,12,49.864410,UBER
